In [ ]:
# Requirements
pip install kserve

In [ ]:
from kserve import RESTConfig, InferenceRESTClient, InferRequest, InferInput
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from random import randrange
import numpy as np
import os
import json
import requests
import io

In [ ]:
# Download dataset
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

In [ ]:
%%sh
unzip -q dogImages.zip
mv dogImages dog-breeds-dataset

In [ ]:
# Dataset
ds_dog_breeds_path = "./dog-breeds-dataset"
test_ds_dog_breeds_path = os.path.join(ds_dog_breeds_path, "test")
ds_dog_breeds = datasets.ImageFolder(test_ds_dog_breeds_path)
dog_breeds_classes = ds_dog_breeds.find_classes(test_ds_dog_breeds_path)[0]

print(f"Classes dog_breeds_classifier: {dog_breeds_classes}")

In [ ]:
# Show sample image
fig = plt.figure(figsize=(1, 1))
sample_idx = randrange(len(ds_dog_breeds))

img, label = ds_dog_breeds[sample_idx]
print(img.format)
fig.add_subplot(1, 1, 1)

plt.title(dog_breeds_classes[label])
plt.axis("off")
plt.imshow(img)
plt.show()

In [ ]:
# Convert image to bytes
def image_to_byte_array(image: Image) -> bytes:
    # BytesIO is a file-like buffer stored in memory
    imgByteArr = io.BytesIO()
    # image.save expects a file-like as a argument
    image.save(imgByteArr, format="jpeg")
    # Turn the BytesIO object back into a bytes object
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr


img_bytes = image_to_byte_array(img)

In [ ]:
# Inference
model_name = "dog-breeds-classifier"
headers = {
    "Content-Type": "image/jpeg", # Or "image/png", "image/webp", etc.
}

base_url = "https://dog-breeds-classifier-modelserving-demo.apps.rosa.n1t3u2f3w1s0b1d.kkw2.p3.openshiftapps.com/"
#base_url = f"https://dog-breeds-classifier-modelserving-demo.apps.rosa.n1t3u2f3w1s0b1d.kkw2.p3.openshiftapps.com/v2/models/dog-breeds-classifier/infer"

In [ ]:
import base64
base64_encoded_data = base64.b64encode(img_bytes).decode("utf-8")
json_data = json.dumps({"image": base64_encoded_data})

image = Image.open(io.BytesIO(img_bytes))

img_array = np.array(img)
        
# Convert the NumPy array to a list to ensure JSON serializability
data = img_array.tolist()

# Encode the data to JSON
json_data = json.dumps(data)

infer_response = requests.post(base_url, headers=headers, data=data)
infer_response.raise_for_status()
print(infer_response.text)
response_data = infer_response.json()
print(json.dumps(response_data, indent=2))


det_class = dog_breeds_classes[np.argmax(infer_response.outputs[0].data)]
# print("Raw response: ", infer_response)
print(f"Detected class: {det_class}")
